In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
import os
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import anndata as ad
import pickle
import scanpy as sc

## Training and Predicting on the remaining images

### Scanorama

In [12]:
markers = ['Ki67', 'IDO', 'GranzymeB', 'GranzymeK', 'CD68', 'HLA-DR', 'IRF4']
mode = 'scanorama'
adata = ad.read_h5ad("/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/standard/adatas/cells_annotated_pp_osteocytes_cleaned_nbh.h5ad")


csv_list = []
for file in os.listdir(f'/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/downstream/non-spatial/thresholding/{mode}'):
    if file.endswith('.csv'):
        csv = pd.read_csv(os.path.join(f'/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/downstream/non-spatial/thresholding/{mode}', file), index_col=0)
        csv_list.append(csv)
df = pd.concat(csv_list)
df.drop(columns=['distance_to_bone', 'X_centroid', 'Y_centroid', 'Phenotype', 'Phenotype2', 'disease', 'image_ID', 'patient_ID', 'Object', 'ROI', ' 1', ' 2', ' 3', ' 4', ' 5', ' 6', 'HistoneH3', '191Ir', '193Ir'], inplace=True)

label_encoder = LabelEncoder()
df['cellcharter_CN'] = label_encoder.fit_transform(df['cellcharter_CN'])
cellcharter_encoder = label_encoder
label_encoder = LabelEncoder()
df['Phenotype3'] = label_encoder.fit_transform(df['Phenotype3'])
phenotype_encoder = label_encoder
label_encoder = LabelEncoder()
df['disease2'] = label_encoder.fit_transform(df['disease2'])
disease_encoder = label_encoder
encoders = {
    'cellcharter': cellcharter_encoder,
    'phenotype3': phenotype_encoder,
    'disease2': disease_encoder
}
for name, encoder in encoders.items():
    with open(f'/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/downstream/non-spatial/thresholding/encoders/{name}_encoder_{mode}.pkl', 'wb') as f:
        pickle.dump(encoder, f)
for marker in markers:
    df2 = df.drop(columns=[x for x in df.columns if 'counts' in x and x != f'{marker}_counts'])
    X = df2.drop(columns=f'{marker}_counts')
    y = df2[f'{marker}_counts']
    classes = np.unique(y)
    weights = compute_class_weight('balanced', classes=classes, y=y)
    class_weights = dict(zip(classes, weights))
    model = XGBClassifier(objective='binary:logistic', eval_metric=['logloss', 'auc'], n_estimators=200, max_depth=6, learning_rate=0.1, tree_method='hist', n_jobs=8, subsample=0.8, scale_pos_weight=class_weights[1] / class_weights[0])
    model.fit(X, y)
    model.save_model(f'/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/downstream/non-spatial/thresholding/xgb_models/xgb_{mode}_{marker}.json')
    diff = set(X.columns) - set(adata.var_names)
    df3 = adata.to_df()
    for col in diff:
        df3[col] = adata.obs[col]
    if set(df3.columns) == set(X.columns):
        df3 = df3[X.columns]
    else:
        print("Columns do not match.")
    df3['cellcharter_CN'] = cellcharter_encoder.transform(df3['cellcharter_CN'])
    df3['Phenotype3'] = phenotype_encoder.transform(df3['Phenotype3'])
    df3['disease2'] = disease_encoder.transform(df3['disease2'])
    adata2 = ad.read_h5ad("/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/standard/adatas/old_preprocessed/cells_scanorama.h5ad") # Previous scanorama corrected values 
    df_scanorama = adata2.to_df()
    df_scanorama.drop(columns=[' 1', ' 2', ' 3', ' 4', ' 5', ' 6', 'HistoneH3', '191Ir', '193Ir'], inplace=True)
    df_scanorama.index = df_scanorama.index.str.replace('tiff', 'csv')
    df_scanorama = df_scanorama[df_scanorama.index.isin(df3.index)]
    columns_match = set(df3.columns).intersection(df_scanorama.columns)
    for column in columns_match:
        if column in df3.columns and column in df_scanorama.columns:
            df3.loc[df_scanorama.index.intersection(df3.index), column] = df_scanorama.loc[df_scanorama.index.intersection(df3.index), column]
    model = XGBClassifier()
    model.load_model(f'/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/downstream/non-spatial/thresholding/xgb_models/xgb_{mode}_{marker}.json')
    pred = model.predict(df3)
    adata.obs[f'{marker}_counts'] = pred

In [5]:
df.isna().sum()

ATP5A                             0
B4GALT1                           0
CD11b                             0
CD11c                             0
CD138                             0
CD3                               0
CD34                              0
CD36                              0
CD38                              0
CD4                               0
CD45                              0
CD68                              0
CD8                               0
CD98                              0
CPT1A                             0
CS                                0
CathepsinK                        0
CollagenTypeI                     0
GLUT1                             0
GranzymeB                         0
GranzymeK                         0
HIF1A                             0
HLA-DR                            0
IDO                               0
IL32                              0
IRF4                              0
Ki67                              0
MPO                         

In [ ]:
adata.obs['CD68_counts'].value_counts()

In [13]:
adata.write_h5ad("/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/standard/adatas/cells_thresholds_scanorama.h5ad")

## Combat

In [14]:
mode = 'combat'
adata = ad.read_h5ad("/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/standard/adatas/cells_annotated_pp_osteocytes_cleaned_nbh.h5ad")
sc.pp.combat(adata, key='patient_ID')

csv_list = []
for file in os.listdir(f'/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/downstream/non-spatial/thresholding/{mode}'):
    if file.endswith('.csv'):
        csv = pd.read_csv(os.path.join(f'/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/downstream/non-spatial/thresholding/{mode}', file), index_col=0)
        csv_list.append(csv)
df = pd.concat(csv_list)
df.drop(columns=['distance_to_bone', 'X_centroid', 'Y_centroid', 'Phenotype', 'Phenotype2', 'disease', 'image_ID', 'patient_ID', 'Object', 'ROI'], inplace=True)
label_encoder = LabelEncoder()
df['cellcharter_CN'] = label_encoder.fit_transform(df['cellcharter_CN'])
cellcharter_encoder = label_encoder
label_encoder = LabelEncoder()
df['Phenotype3'] = label_encoder.fit_transform(df['Phenotype3'])
phenotype_encoder = label_encoder
label_encoder = LabelEncoder()
df['disease2'] = label_encoder.fit_transform(df['disease2'])
disease_encoder = label_encoder
encoders = {
    'cellcharter': cellcharter_encoder,
    'phenotype3': phenotype_encoder,
    'disease2': disease_encoder
}
for name, encoder in encoders.items():
    with open(f'/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/downstream/non-spatial/thresholding/encoders/{name}_encoder_{mode}.pkl', 'wb') as f:
        pickle.dump(encoder, f)
for marker in markers:
    df2 = df.drop(columns=[x for x in df.columns if 'counts' in x and x != f'{marker}_counts'])
    X = df2.drop(columns=f'{marker}_counts')
    y = df2[f'{marker}_counts']
    classes = np.unique(y)
    weights = compute_class_weight('balanced', classes=classes, y=y)
    class_weights = dict(zip(classes, weights))
    model = XGBClassifier(objective='binary:logistic', eval_metric=['logloss', 'auc'], n_estimators=200, max_depth=6, learning_rate=0.1, tree_method='hist', n_jobs=8, subsample=0.8, scale_pos_weight=class_weights[1] / class_weights[0])
    model.fit(X, y)
    model.save_model(f'/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/downstream/non-spatial/thresholding/xgb_models/xgb_{mode}_{marker}.json')
    diff = set(X.columns) - set(adata.var_names)
    df3 = adata.to_df()
    for col in diff:
        df3[col] = adata.obs[col]
    if set(df3.columns) == set(X.columns):
        df3 = df3[X.columns]
    else:
        print("Columns do not match.")
    df3['cellcharter_CN'] = cellcharter_encoder.transform(df3['cellcharter_CN'])
    df3['Phenotype3'] = phenotype_encoder.transform(df3['Phenotype3'])
    df3['disease2'] = disease_encoder.transform(df3['disease2'])
    model = XGBClassifier()
    model.load_model(f'/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/downstream/non-spatial/thresholding/xgb_models/xgb_{mode}_{marker}.json')
    pred = model.predict(df3)
    adata.obs[f'{marker}_counts'] = pred

In [15]:
adata.write_h5ad("/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/standard/adatas/cells_thresholds_combat.h5ad")

## Only scaling

In [9]:
mode = 'scaled'
adata = ad.read_h5ad("/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/standard/adatas/cells_annotated_pp_osteocytes_cleaned_nbh.h5ad")

csv_list = []
for file in os.listdir(f'/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/downstream/non-spatial/thresholding/{mode}'):
    if file.endswith('.csv'):
        csv = pd.read_csv(os.path.join(f'/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/downstream/non-spatial/thresholding/{mode}', file), index_col=0)
        csv_list.append(csv)
df = pd.concat(csv_list)
df.drop(columns=['distance_to_bone', 'X_centroid', 'Y_centroid', 'Phenotype', 'Phenotype2', 'disease', 'image_ID', 'patient_ID', 'Object', 'ROI'], inplace=True)
label_encoder = LabelEncoder()
df['cellcharter_CN'] = label_encoder.fit_transform(df['cellcharter_CN'])
cellcharter_encoder = label_encoder
label_encoder = LabelEncoder()
df['Phenotype3'] = label_encoder.fit_transform(df['Phenotype3'])
phenotype_encoder = label_encoder
label_encoder = LabelEncoder()
df['disease2'] = label_encoder.fit_transform(df['disease2'])
disease_encoder = label_encoder
encoders = {
    'cellcharter': cellcharter_encoder,
    'phenotype3': phenotype_encoder,
    'disease2': disease_encoder
}
for name, encoder in encoders.items():
    with open(f'/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/downstream/non-spatial/thresholding/encoders/{name}_encoder_{mode}.pkl', 'wb') as f:
        pickle.dump(encoder, f)
for marker in markers:
    df2 = df.drop(columns=[x for x in df.columns if 'counts' in x and x != f'{marker}_counts'])
    X = df2.drop(columns=f'{marker}_counts')
    y = df2[f'{marker}_counts']
    classes = np.unique(y)
    weights = compute_class_weight('balanced', classes=classes, y=y)
    class_weights = dict(zip(classes, weights))
    model = XGBClassifier(objective='binary:logistic', eval_metric=['logloss', 'auc'], n_estimators=200, max_depth=6, learning_rate=0.1, tree_method='hist', n_jobs=8, subsample=0.8, scale_pos_weight=class_weights[1] / class_weights[0])
    model.fit(X, y)
    model.save_model(f'/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/downstream/non-spatial/thresholding/xgb_models/xgb_{mode}_{marker}.json')
    diff = set(X.columns) - set(adata.var_names)
    df3 = adata.to_df()
    for col in diff:
        df3[col] = adata.obs[col]
    if set(df3.columns) == set(X.columns):
        df3 = df3[X.columns]
    else:
        print("Columns do not match.")
    df3['cellcharter_CN'] = cellcharter_encoder.transform(df3['cellcharter_CN'])
    df3['Phenotype3'] = phenotype_encoder.transform(df3['Phenotype3'])
    df3['disease2'] = disease_encoder.transform(df3['disease2'])
    model = XGBClassifier()
    model.load_model(f'/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/downstream/non-spatial/thresholding/xgb_models/xgb_{mode}_{marker}.json')
    pred = model.predict(df3)
    adata.obs[f'{marker}_counts'] = pred

KeyboardInterrupt: 